## $\color{Green}{\text{GMD Table 4: GMSL from Workflows}}$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### $\color{blue}{\text{List of Modules and Data to be used}}$

In [2]:
# Load FACTS workflow data set names as a dataframe.
df_WF = pd.read_fwf( '001_IP_GMD_Table_names_Modules_Data/mod-submod-data_WF_global_Bob_amarel_V2.txt' )
df_WF


,Component,Workflow,Data_File
0,total,wf1e,total.workflow.wf1e.global.nc
1,total,wf1f,total.workflow.wf1f.global.nc
2,total,wf2e,total.workflow.wf2e.global.nc
3,total,wf2f,total.workflow.wf2f.global.nc
4,total,wf3e,total.workflow.wf3e.global.nc
5,total,wf3f,total.workflow.wf3f.global.nc
6,total,wf4,total.workflow.wf4.global.nc


#### $\color{Red}{\text{Load Module data from ==> facts-experiments}}$

In [3]:
EXPDATE     = 221217
MODNO       = df_WF.index.values;   # Number of datasets Module outputs
# SCENARIO    = ['ssp126', 'ssp245','ssp585'];
SCENARIO    = ['ssp119','ssp126', 'ssp245', 'ssp370', 'ssp585'];
YEAR        = 2100; # Percentile Year for the table.   
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#        
for val in MODNO:
    for scenario in SCENARIO: 
        WF = df_WF["Workflow"][val];  COMP = df_WF["Component"][val];  
        # Skip if data is absent.
        if df_WF["Data_File"][val] == 'XXX':
           continue 
        #
        if (scenario == 'ssp119') & ((WF == 'wf3e') or (WF == 'wf3f') or (WF == 'wf4')):
            continue
        if (scenario == 'ssp370') & ((WF == 'wf3e') or (WF == 'wf3f') or (WF == 'wf4')):
            continue
        if (scenario == 'ssp245') & ((WF == 'wf3e') or (WF == 'wf3f')):
            continue
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/coupling.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario) + df_WF["Data_File"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        Yind    = np.where(d_nc["years"].values == YEAR)[0][0]
        GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/1000  # Convert to cmeters.
        # Find Percentiles.
        pcntle  = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        TEMP = [COMP,WF, scenario] + pcntle.tolist()
        df_TEMP.append( TEMP )
df_ptile = pd.DataFrame( df_TEMP, columns=['Component','Workflow','SSP', ] + [ f'col_{x}' for xi, x in enumerate( percentList )] )
#
df_ptile[''] = df_ptile.apply(lambda x: f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', axis=1 )
df_ptile1    = pd.DataFrame( df_ptile.set_index( ['Component', 'Workflow', 'SSP'] )[''] ).unstack().swaplevel( 0,1, axis=1 )
print('Percentile of GMSL [ median(17th,83rd) ] in cm for different workflows for the year ', YEAR)
df_ptile1


Percentile of GMSL [ median(17th,83rd) ] in cm for different workflows for the year  2100


SSP                             ssp119              ssp126  \
                                                             
Component Workflow                                           
total     wf1e      0.35 (0.27 , 0.44)  0.40 (0.32 , 0.49)   
          wf1f      0.35 (0.27 , 0.44)  0.40 (0.31 , 0.49)   
          wf2e      0.40 (0.30 , 0.53)  0.46 (0.35 , 0.60)   
          wf2f      0.41 (0.33 , 0.54)  0.48 (0.38 , 0.62)   
          wf3e                     NaN  0.40 (0.34 , 0.48)   
          wf3f                     NaN  0.43 (0.37 , 0.49)   
          wf4                      NaN  0.53 (0.37 , 0.80)   

SSP                             ssp245              ssp370              ssp585  
                                                                                
Component Workflow                                                              
total     wf1e      0.50 (0.42 , 0.60)  0.62 (0.53 , 0.73)  0.71 (0.61 , 0.82)  
          wf1f      0.49 (0.40 , 0.59)  0.58 (0.48 , 0.68)  0.66 (0.55 , 0.78)  
          wf2e      0.57 (0.45 , 0.73)  0.70 (0.57 , 0.88)  0.80 (0.65 , 1.00)  
          wf2f      0.59 (0.48 , 0.74)  0.70 (0.58 , 0.87)  0.80 (0.66 , 1.00)  
          wf3e                     NaN                 NaN  0.97 (0.80 , 1.18)  
          wf3f                     NaN                 NaN  0.97 (0.81 , 1.17)  
          wf4       0.60 (0.44 , 0.88)                 NaN  1.01 (0.69 , 1.64)

In [4]:
print(df_ptile1.to_latex(index=True, escape=False))

\begin{tabular}{lllllll}
\toprule
      & SSP &              ssp119 &              ssp126 &              ssp245 &              ssp370 &              ssp585 \\
      & {} \\
Component & Workflow &                     &                     &                     &                     &                     \\
\midrule
total & wf1e &  0.35 (0.27 , 0.44) &  0.40 (0.32 , 0.49) &  0.50 (0.42 , 0.60) &  0.62 (0.53 , 0.73) &  0.71 (0.61 , 0.82) \\
      & wf1f &  0.35 (0.27 , 0.44) &  0.40 (0.31 , 0.49) &  0.49 (0.40 , 0.59) &  0.58 (0.48 , 0.68) &  0.66 (0.55 , 0.78) \\
      & wf2e &  0.40 (0.30 , 0.53) &  0.46 (0.35 , 0.60) &  0.57 (0.45 , 0.73) &  0.70 (0.57 , 0.88) &  0.80 (0.65 , 1.00) \\
      & wf2f &  0.41 (0.33 , 0.54) &  0.48 (0.38 , 0.62) &  0.59 (0.48 , 0.74) &  0.70 (0.58 , 0.87) &  0.80 (0.66 , 1.00) \\
      & wf3e &                 NaN &  0.40 (0.34 , 0.48) &                 NaN &                 NaN &  0.97 (0.80 , 1.18) \\
      & wf3f &                 NaN &  0.43 (0.37 , 0.4

/tmp/ipykernel_40857/122110213.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_ptile1.to_latex(index=True, escape=False))
